In [ ]:
import sys
from os import listdir
from os.path import isfile, join
import datetime

import scipy as sy
import scipy.fftpack as syfp
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mtpl
import numpy as np

from pyapnea.oscar.oscar_loader import load_session
from pyapnea.oscar.oscar_getter import event_data_to_dataframe
from pyapnea.oscar.oscar_constants import CHANNELS, ChannelID

from src.data.preparation_tasks import align_channels

In [ ]:
filename_to_load = '../data/raw/ResMed_23192565579/Events/62202198.001'
oscar_session_data = load_session(filename_to_load)
df = event_data_to_dataframe(oscar_session_data, [ChannelID.CPAP_FlowRate.value, ChannelID.CPAP_ClearAirway.value, ChannelID.CPAP_Obstructive.value])
df

In [ ]:
df[df['FlowRate'].isnull()]

In [ ]:
df_aligned = align_channels(df, 'FlowRate', '40ms')
df_aligned.set_index('time_utc', inplace=True)
df_aligned

In [ ]:
df_aligned[df_aligned['FlowRate'].isnull()]

In [ ]:
dfc = df_aligned[['FlowRate']]
# dfm = dfc.melt('time_utc', var_name='cols', value_name='vals')
# dfm.sort_values(by=['time_utc'], inplace=True, ignore_index=True)
# dfm_annotation = dfm[(~pd.isnull(dfm['vals']) & ((dfm['cols']=='Obstructive') | (dfm['cols']=='ClearAirway')))]
# display(dfm_annotation)
dfm_annotation=df_aligned[~df_aligned['Obstructive'].isna() | ~df_aligned['ClearAirway'].isna()]
dfm_annotation=dfm_annotation[['Obstructive', 'ClearAirway']]
dfm_annotation

In [ ]:
event_time = dfm_annotation.index[1]
_ = plt.figure(figsize=(25,10))
_ = plt.plot(df_aligned['FlowRate'], color='blue', label='Original')
_ = plt.plot(dfm_annotation[~dfm_annotation['Obstructive'].isna()], linestyle='none', marker='X', color='red', markersize=10)
_ = plt.plot(dfm_annotation[~dfm_annotation['ClearAirway'].isna()], linestyle='none', marker='X', color='green', markersize=10)
_ = plt.legend(loc='best')
_ = plt.title('FlowRate')
plt.ylim(-50, 50)
plt.xlim(event_time - datetime.timedelta(seconds=50), event_time + datetime.timedelta(seconds=10))
plt.show()